### TFIDF -> Random Forest Regressor

#### Nessecary Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np

#### Load in data / Separate out/ drop numerical data, isolate text data.

In [2]:
train_df = pd.read_csv('checkpoint_train.csv', encoding = 'utf-8')
test_df = pd.read_csv('checkpoint_test.csv', encoding = 'utf-8')

In [3]:
text_columns = ['overview', 'tagline', 'title', 'all_keywords', 'part_of_collection']
numerical_columns = ['userId', 'movieId', 'release_date', 'budget', 'popularity', 'revenue', 
                     'runtime', 'vote_average', 'vote_count', 'old', 
                     'Science Fiction', 'Crime', 'Thriller', 'Drama', 
                     'Horror', 'Romance', 'TV Movie', 'Action', 
                     'Adventure', 'War', 'Documentary', 'Family', 
                     'Fantasy', 'Music', 'Animation', 'Mystery', 
                     'Comedy', 'Western', 'Foreign', 'History']

#### Create combined text column

In [5]:
train_df[text_columns] = train_df[text_columns].fillna('').astype(str)
test_df[text_columns] = test_df[text_columns].fillna('').astype(str)

In [6]:
train_df['corpus'] = train_df['overview'] + "_" + train_df['tagline'] + "_" + train_df['title'] + "_" + train_df['all_keywords'] + "_" + train_df['part_of_collection']
test_df['corpus'] = test_df['overview'] + "_" + test_df['tagline'] + "_" + test_df['title'] + "_" + test_df['all_keywords'] + "_" + test_df['part_of_collection']

#### Initialize Vectorizer, fit to text column, create tfidf matricies

In [7]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['corpus'])
train_tfidf_matrix = vectorizer.transform(train_df['corpus'])
test_tfidf_matrix = vectorizer.transform(test_df['corpus'])

#### Create train/ test sets

In [8]:
y_train = train_df['rating']
X_train = train_tfidf_matrix

X_test = test_tfidf_matrix

#### Initialize/ Train Lasso Model

In [9]:
lasso_regressor = Lasso(alpha = .01)

In [10]:
lasso_regressor.fit(X_train, y_train)

Lasso(alpha=0.01)

#### Save / Submit Results

In [11]:
test_df['rating'] = (lasso_regressor.predict(X_test))
test_df['movieId'] = test_df['movieId'].astype(int).astype(str)
test_df['userId'] = test_df['userId'].astype(int).astype(str)
test_df['userId_movieId'] = test_df['userId'] + '_' + test_df['movieId']
cols = ['userId_movieId', 'rating']
test_df = test_df[cols]

In [13]:
test_df.to_csv('submission.csv', index = False)

Model RMSE: 0.2142